In [2]:
#chack for updated files\n,
%load_ext autoreload
%autoreload 2
#import packages
from __future__ import division

import numpy as np
import glob, json
import pickle
import os as os_pack
import matplotlib.pyplot as plt
import corner
#%matplotlib inline\n",
%config InlineBackend.figure_format = 'retina'
import healpy as hp
import os, glob, json, pickle
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as sl
import enterprise
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
import enterprise.constants as const
from enterprise_extensions import blocks
from enterprise_extensions import models as ee_models
from enterprise_extensions import model_utils as ee_model_utils
from enterprise_extensions import model_orfs
from enterprise_extensions.frequentist import optimal_statistic as opt_stat
from enterprise_extensions import sampler as ee_sampler
from enterprise.signals.signal_base import LogLikelihood
import enterprise_wavelets as models
from enterprise.signals.deterministic_signals import Deterministic
from enterprise.signals.parameter import function
from la_forge.core import Core
from la_forge.diagnostics import plot_chains
from la_forge import rednoise
import la_forge
import corner
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc
    #style

In [12]:
#Loading in pickle and noise files
pint_pickle = '/home/gia/Jacob_Taylor/15yr_v1.1/Data/PINT/v1p1_de440_pint_bipm2019.pkl'
#load noise dictionary
noise_file = '/home/gia/Jacob_Taylor/15yr_v1.1/Data/PINT/v1p1_wn_dict.json'
#psrlist = np.loadtxt('/home/reyna/15yr_v1p0/15yr_v1-20211001T235643Z-001/15yr_v1/psrlist_15yr_pint.txt', dtype = str)
with open(noise_file, 'r') as h:
    noise_params = json.load(h)
with open(pint_pickle,'rb') as f:
    allpsrs = pickle.load(f)
psrs = []
for ii,p in enumerate(allpsrs):
    psrs.append(p)
#Temporary to get code to not crash
psrs = psrs[0:4]
psrlist = [psr.name for psr in psrs]
print(psrlist)
for i in range(len(psrs)):
    print((max(psrs[i].toas) - min(psrs[i].toas))/(3.17*10**(7)))

['B1855+09', 'B1937+21', 'B1953+29', 'J0023+0923']
15.515060297068947
15.802872967471835
11.08458774068679
8.97542529762611


In [20]:
glitches = []
log10_f0 = parameter.Uniform(np.log10(3.5e-9), np.log10(1e-7))("Glitch_"+str(1)+'_'+'log10_f0')
phase0 = parameter.Uniform(0, 2*np.pi)("Glitch_"+str(1)+'_'+'phase0')
tau = parameter.Uniform(0.2, 5)("Glitch_"+str(1)+'_'+'tau')
t0 = parameter.Uniform(0.0, 10.0)("Glitch_"+str(1)+'_'+'t0')
psr_idx = parameter.Uniform(-0.5, len(psrs)-0.5)("Glitch_"+str(1)+'_'+'psr_idx')
log10_h = parameter.LinearExp(-6.5, -5)("Glitch_"+str(1)+'_'+'log10_h')
glitch_wf = models.glitch_delay(log10_h = log10_h, tau = tau, log10_f0 = log10_f0, t0 = t0, phase0 = phase0, tref=53000*86400,
                                        psr_float_idx = psr_idx, pulsars=psrs)
glitches.append(deterministic_signals.Deterministic(glitch_wf, name='Glitch'+str(1) ))

In [21]:
tm = gp_signals.TimingModel(use_svd=True)
wn = blocks.white_noise_block(vary=False, inc_ecorr=False)
#s = base_model
s = tm + wn + glitches[0]
model = []
for p in psrs:
    model.append(s(p))
with open(noise_file, 'r') as fp:
    noisedict = json.load(fp)
    pta = signal_base.PTA(model)
    pta.set_default_params(noisedict)

In [22]:
d0 = parameter.sample(pta.params)
x0 = np.array([d0[par.name] for par in pta.params])

In [23]:
pta.get_lnlikelihood(x0)

phiinv_method  cliques
resres_logdet  -24150738.692848988
resres_logdet+logsigprior -24150738.692848988
resres_logdet+logsigprior+Tnr_sigma_phi2 -24157521.871626034
logdet_phi:  15289.165017480458
logdet_sigma 4448.603620972838
resres_logdet+logsigprior+Tnr_sigma_phi2 292923.58086034283
logdet_phi:  25604.746234093782
logdet_sigma 8639.86438247935
resres_logdet+logsigprior+Tnr_sigma_phi2 287011.1716961497
logdet_phi:  10683.994831492373
logdet_sigma 2880.4602861881
resres_logdet+logsigprior+Tnr_sigma_phi2 280157.0007524602
logdet_phi:  10591.89142777261
logdet_sigma 3117.0202293660377


280157.0007524602

In [57]:
285302.890035331
logdet_phi:  15289.165017480458
logdet_sigma 4448.60362097284
logdet_phi:  25604.746234093782
logdet_sigma 8639.86438247935
logdet_phi:  10683.994831492373
logdet_sigma 2880.4602861881012
logdet_phi:  10591.89142777261
logdet_sigma 3117.0202293660386

441415.18743292545

In [113]:
pta.params

[Glitch_1_log10_f0:Uniform(pmin=-8.455931955649724, pmax=-7.0),
 Glitch_1_log10_h:LinearExp(pmin=-10, pmax=-9),
 Glitch_1_phase0:Uniform(pmin=0, pmax=6.283185307179586),
 Glitch_1_psr_idx:Uniform(pmin=-0.5, pmax=66.5),
 Glitch_1_t0:Uniform(pmin=0.0, pmax=10.0),
 Glitch_1_tau:Uniform(pmin=0.2, pmax=5)]

In [26]:
print(pta.get_residuals())

[array([-1.57059796e-06, -1.71515979e-06, -8.36003331e-07, ...,
        2.22050960e-06,  1.93448212e-06,  2.50537850e-06]), array([7.58853171e-06, 7.79146086e-06, 7.64545029e-06, ...,
       7.36077832e-06, 6.86636193e-06, 6.64400339e-06]), array([-1.70031432e-05, -1.55066449e-05, -1.01133242e-05, ...,
       -5.89991309e-06, -5.51676409e-09, -1.21392684e-06]), array([ 8.00383849e-07, -1.27680358e-07,  1.04411788e-06, ...,
       -6.89742274e-07,  1.33288660e-06,  6.94847330e-06])]


In [27]:
pta.param_names

['Glitch_1_log10_f0',
 'Glitch_1_log10_h',
 'Glitch_1_phase0',
 'Glitch_1_psr_idx',
 'Glitch_1_t0',
 'Glitch_1_tau']

In [29]:
x0[3]

0.5371120001653811

In [35]:
import Fast_Burst_likelihood_ent_version as FB_ent
import Fast_Burst_likelihood as FB

In [36]:
FB1 = FB_ent.FastBurst(pta = pta, psrs = psrs, params = d0, Npsr = len(psrs), tref=53000*86400)
#FB2 = FB.FastBurst(pta = pta, psrs = psrs, params = d0, Npsr = len(psrs), tref=53000*86400)

Nvecs arary:  [[1.69852389e-12 1.12772641e-10 3.26328092e-12 ... 4.85655692e-12
 5.14669141e-13 1.79238956e-12], [6.95185145e-14 1.57784257e-13 2.75345324e-14 ... 1.37317696e-13
 6.53510393e-13 3.42658995e-13], [4.94236756e-10 4.03404993e-10 7.90600715e-10 ... 2.03923984e-10
 4.69194849e-10 4.02505715e-10], [1.78150121e-13 6.22555077e-13 1.41224754e-13 ... 4.44123286e-11
 2.71409648e-11 3.12885720e-11], ...]
-24150738.692848988
logdet_phi:  15289.165017480458
logdet_sigma 4448.60362097284
logdet_phi:  25604.746234093782
logdet_sigma 8639.86438247935
logdet_phi:  10683.994831492373
logdet_sigma 2880.4602861881012
logdet_phi:  10591.89142777261
logdet_sigma 3117.0202293660386
287678.36202822253


In [37]:
 475725.0537348674

475725.0537348674

In [38]:
FB1.get_lnlikelihood(10**(x0[1]), x0[2], 10**(x0[0]), (3.15*10**7)*x0[5], (3.15*10**7)*x0[4], x0[3])

Amplitude:  1.2855510128935444e-06
Frequency:  5.67798550892742e-09
glitch_index:  0.5371120001653811
Old Sigma:  [0. 0.]
New sigma:  [-3.14046587e-07 -1.24660184e-06]
Old M and N:  [[0. 0.]
 [0. 0.]] [0. 0.]
Input time:  433.22629556269334
Per pulsar toas:  [ 358.72367538  358.72367538  358.72367538 ... 6051.17056678 6051.17056678
 6051.17056678]
Cosine:  [9.67138218e-01 9.67138218e-01 9.67138218e-01 ... 5.78261299e-19
 5.78261299e-19 5.78261299e-19]
Sine:  [-2.26088185e-01 -2.26088185e-01 -2.26088185e-01 ... -1.52310391e-17
 -1.52310391e-17 -1.52310391e-17]
Exponential:  [-3436.67262066 -3436.67262066 -3436.67262066 ... -4215.69744073
 -4215.69744073 -4215.69744073]
Per pulsar toas:  [ 267.08925417  267.08925417  267.08925418 ... 6065.13408132 6065.13408135
 6065.13408135]
Cosine:  [8.42691845e-01 8.42691845e-01 8.42691845e-01 ... 1.01692897e-18
 1.01692897e-18 1.01692897e-18]
Sine:  [-4.73692570e-01 -4.73692570e-01 -4.73692570e-01 ... -1.25289526e-17
 -1.25289526e-17 -1.25289526e-17

285302.890035331

In [33]:
FB2.get_lnlikelihood(10**(x0[1]), x0[2], 10**(x0[0]), (3.15*10**7)*x0[5], (3.15*10**7)*x0[4], x0[3])

Amplitude:  4.258573214906627e-06
Frequency:  9.818791332648557e-08
glitch_index:  0.02993512229379247
Old Sigma:  [0. 0.]
New sigma:  [-3.69111611e-07 -4.24254669e-06]
Old M and N:  [[0. 0.]
 [0. 0.]] [0. 0.]
Input time:  592.8745928115762
Per pulsar toas:  [ 394.85368635  394.85368641  394.85368641 ... 6067.37530852 6067.37530852
 6067.37530852]
Cosine:  [-4.20982018e-01 -4.20982021e-01 -4.20982021e-01 ... -5.70972603e-05
 -5.70972603e-05 -5.70972603e-05]
Sine:  [8.93140714e-01 8.93140713e-01 8.93140713e-01 ... 2.15445657e-05
 2.15445657e-05 2.15445657e-05]
Exponential:  [ -902.75732599  -902.75732599  -902.75732599 ... -1107.14286197
 -1107.14286197 -1107.14286197]
Per pulsar toas:  [2972.08141782 2972.08141786 2972.08141793 ... 5942.92773218 5942.92773218
 5942.92773222]
Cosine:  [ 6.45851716e-02  6.45851713e-02  6.45851707e-02 ... -7.15209358e-05
 -7.15209358e-05 -7.15209359e-05]
Sine:  [1.46332714e-01 1.46332714e-01 1.46332714e-01 ... 6.16057866e-05
 6.16057866e-05 6.16057865e-05

482127.23089051904

In [34]:
482127.23089051904 #Bence version

482127.23089051904

In [35]:
448629.5379522237 #Enterprise version

448629.5379522237